In [24]:
import tensorflow as tf
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

In [25]:
"""
Global values
"""
FEATURE_LEN = 20
folder = 'year2010_movie'


In [26]:

user_rating = pickle.load(open('../data/'+folder+'/user_rating.pkl', 'rb'))

user_rating_valid = pickle.load(open('../data/'+folder+'/user_rating_validation.pkl', 'rb'))

user_rating_test = pickle.load(open('../data/'+folder+'/user_rating_test.pkl', 'rb'))

df_train = []
max_user = 0
max_movie = 0
for user in user_rating:
    user_i = int(user)
    for tup in user_rating[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_train.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i
        
df_test = []
for user in user_rating_test:
    user_i = int(user)
    for tup in user_rating_test[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_test.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i
        

df_valid = []
for user in user_rating_valid:
    user_i = int(user)
    for tup in user_rating_valid[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_valid.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i

#user_s, movie_s, rating_s = shuffle(user_list, movie_list, rating_list)

#df_train, df_test = train_test_split(df, test_size = 0.2)

"""
"""
user_s = [t[0] for t in df_train]
movie_s = [t[1] for t in df_train]
rating_s = [t[2] for t in df_train]

user_s_test = [t[0] for t in df_test]
movie_s_test = [t[1] for t in df_test]
rating_s_test = [t[2] for t in df_test]

user_s_valid = [t[0] for t in df_valid]
movie_s_valid = [t[1] for t in df_valid]
rating_s_valid = [t[2] for t in df_valid]

print("training number: ", len(user_s))
print("validaton number: ", len(user_s_valid))
print("test number: ", len(user_s_test))

training number:  201623
validaton number:  25203
test number:  25203


In [27]:
user_indecies_train = user_s
item_indecies_train = movie_s
rates_train = rating_s

user_indecies_test = user_s_test
item_indecies_test = movie_s_test
rates_test = rating_s_test

user_indecies_valid = user_s_valid
item_indecies_valid = movie_s_valid
rates_valid = rating_s_valid


In [28]:
max_movie

3906

In [29]:
totle_len = len(user_s)
totle_len

201623

In [30]:
def get_bias(user, movie, rating):
    print('max user:', max_user)
    print('max movie:', max_movie)
    average = np.mean(rating)
    user_dict = {}
    movie_dict = {}
    for i in range(len(user)):
        user_dict[user[i]] = [rating[i]] if user[i] not in user_dict else user_dict[user[i]] + [rating[i]]
        movie_dict[movie[i]] = [rating[i]] if movie[i] not in movie_dict else movie_dict[movie[i]] + [rating[i]]
    
    user_bias = [0 for i in range(max_user + 1)]
    movie_bias = [0 for i in range(max_movie + 1)]
    print(user_dict[2253])
    print(len(user_bias), len(movie_bias))
    for key in user_dict:
        user_bias[int(key)] = np.mean(user_dict[key]) - average
    for key in movie_dict:
        movie_bias[int(key)] = np.mean(movie_dict[key]) - average
    
    return user_bias, movie_bias, average

user_bias, movie_bias, average_score = get_bias(user_s, movie_s, rating_s)


    

max user: 3838
max movie: 3906
[3.5, 3.0, 4.0, 4.0, 4.0, 4.5, 4.0, 2.0, 4.5, 4.0, 2.0, 4.0, 4.5, 4.0, 4.0, 4.0, 4.0, 4.0, 2.0]
3839 3907


In [31]:
user_bias_r = np.reshape(user_bias,[-1,1])
movie_bias_r = np.reshape(movie_bias,[1,-1])

In [32]:
U = tf.Variable(initial_value=tf.truncated_normal([max_user+1, FEATURE_LEN], stddev=0.2, mean=0), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, max_movie+1], stddev=0.2, mean=0), name='movies')

In [33]:

U_plus_bias = tf.concat([U, tf.constant(user_bias_r, dtype=tf.float32, name="user_bias"), tf.ones((max_user+1,1), dtype=tf.float32, name="user_bias_ones")], 1)
P_plus_bias = tf.concat([P, tf.ones((1, max_movie+1), name="movie_bias_ones", dtype=tf.float32), tf.constant(movie_bias_r, dtype=tf.float32, name="movie_bias")], 0)


In [34]:
#result = tf.matmul(U_plus_bias, P_plus_bias)
result = tf.matmul(U, P)

In [35]:
result_flatten = tf.reshape(result, [-1])
R = tf.gather(result_flatten, user_s * tf.shape(result)[1] + movie_s, name='extracting_user_rate')

In [36]:
diff_op = tf.subtract(R, rating_s, name='trainig_diff')
diff_op_squared = tf.nn.l2_loss(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

In [37]:
lda = tf.constant(0.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.nn.l2_loss(U, name='user_abs'), name='user_norm'), 
   tf.reduce_sum(tf.nn.l2_loss(P, name='item_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [38]:
clip_U = U.assign(tf.maximum(tf.zeros_like(U), U))
clip_P = P.assign(tf.maximum(tf.zeros_like(P), P))
clip = tf.group(clip_U, clip_P)

In [39]:
cost = tf.add(base_cost, regularizer)
#cost = base_cost

In [40]:
lr = tf.constant(.0001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.98, staircase=True)
#learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [41]:


# accuracy
R_test = tf.gather(result_flatten, user_indecies_train * tf.shape(result)[1] + item_indecies_train, name='extracting_user_rate_test')
#R_test = tf.cast(R_test, tf.float64)
rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_train, R_test))))
#print(sess.run(rmse))


In [42]:
for i in user_indecies_train:
    if i < 0:
        print(i)
        break

In [43]:


# accuracy
test_R_test = tf.gather(result_flatten, user_indecies_test * tf.shape(result)[1] + item_indecies_test, name='extracting_user_rate_test')
#test_R_test = tf.cast(test_R_test, tf.float64)
test_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_test, test_R_test))))
#print(sess.run(rmse))



In [44]:
# accuracy
valid_R_test = tf.gather(result_flatten, user_indecies_valid * tf.shape(result)[1] + item_indecies_valid, name='extracting_user_rate_valid')
#test_R_test = tf.cast(test_R_test, tf.float64)
valid_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_valid, valid_R_test))))
#print(sess.run(rmse))

In [45]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
minVR = 999
not_change_n = 0
for i in range(10001):
    _, c, r, tr, vr = sess.run([training_step, cost, rmse, test_rmse, valid_rmse])
    sess.run(clip)
    if i%20 == 0:
        print(i, c, r, vr, tr, not_change_n)
    
        if vr + 0.001 < minVR:
            minVR = vr
            not_change_n = 0
        else:
            not_change_n += 1

        if not_change_n > 10:
            break
            
    
    """
    if i%500 == 0:
        r, tr = sess.run([rmse, test_rmse])
        #r = sess.run([rmse])
        print("rmse:", i, r, tr)
    """
    
    

0 1.29791e+06 3.58812 3.59437 3.58403 0
20 633931.0 2.50764 2.52474 2.51376 0
40 298321.0 1.72022 1.73898 1.74205 0
60 202012.0 1.41556 1.43885 1.44342 0
80 158655.0 1.25448 1.28131 1.28641 0
100 134681.0 1.15581 1.18528 1.19068 0
120 119535.0 1.08888 1.12051 1.12615 0
140 109072.0 1.04013 1.07365 1.07954 0
160 101383.0 1.0028 1.03801 1.04416 0
180 95472.1 0.97312 1.00989 1.01629 0
200 90768.8 0.948845 0.987066 0.9937 0
220 86924.1 0.928529 0.968118 0.974969 0
240 83711.8 0.911207 0.952107 0.959153 0
260 80978.6 0.896208 0.938378 0.945594 0
280 78617.0 0.883042 0.926465 0.933823 0
300 76549.5 0.871351 0.91602 0.923499 0
320 74718.4 0.860863 0.906783 0.914363 0
340 73080.2 0.851374 0.89855 0.906213 0
360 71600.7 0.842709 0.891168 0.898895 0
380 70253.2 0.83474 0.884509 0.892283 0
400 69016.9 0.827361 0.878469 0.886284 0
420 67873.7 0.820478 0.87296 0.880805 0
440 66810.5 0.814026 0.867919 0.875784 0
460 65814.5 0.807935 0.863284 0.871162 0
480 64875.6 0.802149 0.859007 0.866888 0
500 63

In [46]:
train_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_train, R_test)))
test_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_test, test_R_test)))
valid_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_valid, valid_R_test)))
mae_tr, mae_v, mae_t = sess.run([train_mae, valid_mae, test_mae])
print(mae_tr)
print(mae_v)
print(mae_t)

0.46849
0.591013
0.593347
